In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
#Run the command to generate the train and test pkl files
command = "python preprocess_cellline.py"
os.system(command)

(337, 824)
(183, 824)
(460, 374)
(211, 374)
['dbgap_rnaseq_sample', 'LASP1', 'HOXA11', 'CREBBP', 'ETV1', 'GAS7', 'CD79B', 'BTK', 'BRCA1', 'WAS', 'WWTR1', 'CD74', 'BIRC3', 'FAS', 'BCLAF1', 'ANK1', 'RABEP1', 'ZCCHC8', 'CUL3', 'FLT4', 'CDH1', 'TNC', 'CTNNA1', 'PREX2', 'TPR', 'GOPC', 'TNFRSF17', 'SNX29', 'ELN', 'ARID1B', 'HERPUD1', 'POLQ', 'PIK3CB', 'THRAP3', 'KMT2C', 'PRDM1', 'POLD1', 'CASP8', 'PMS1', 'NTHL1', 'ERBB3', 'SPEN', 'MAP2K4', 'SMARCD1', 'GOLGA5', 'FGFR2', 'KLF6', 'RHOA', 'CBFB', 'FGFR3', 'TFE3', 'BCL3', 'CLTCL1', 'FSTL3', 'PABPC1', 'TCF3', 'PRKACA', 'TFRC', 'AFF4', 'SMC1A', 'FCGR2B', 'TP63', 'SDHA', 'SMARCE1', 'KDM5A', 'IGF2BP2', 'MAP3K13', 'PICALM', 'EED', 'ARHGEF10L', 'MLH1', 'NT5C2', 'NFKB2', 'FGFR1', 'ARAF', 'N4BP2', 'HOXA9', 'MLLT10', 'PCM1', 'CIC', 'DNM2', 'KEAP1', 'HSP90AA1', 'PTPRC', 'ERC1', 'XPO1', 'PALB2', 'KAT6A', 'CYLD', 'FAT1', 'NCOA1', 'BCORL1', 'ATRX', 'MECOM', 'EPS15', 'BAX', 'GNAS', 'CNOT3', 'GNA11', 'FUS', 'CD209', 'FH', 'ESR1', 'EZR', 'MSH2', 'MAP3K1', 'HSP90

0

In [3]:
#Load the training and test set with gene expression, clinical traits, pathway activations, celltype and module activations, mutations
train_feature_df = pd.read_pickle("../Data/Training_Set_Mod.pkl",compression="zip")
test_feature_df = pd.read_pickle("../Data/Test_Set_Mod.pkl",compression="zip")
print(train_feature_df.shape)
print(test_feature_df.shape)
print(train_feature_df.columns)

(337, 1131)
(183, 1131)
Index(['dbgap_rnaseq_sample', 'LASP1', 'HOXA11', 'CREBBP_x', 'ETV1', 'GAS7',
       'CD79B', 'BTK', 'BRCA1', 'WAS',
       ...
       'frameshift_variant', 'missense_variant', 'stop_gained',
       'inframe_deletion', 'protein_altering_variant',
       'splice_acceptor_variant', 'splice_donor_variant', 'start_lost',
       'inframe_insertion', 'stop_lost'],
      dtype='object', length=1131)


In [4]:
#Load the training and test drug, cell combination file
train_drug_cell_df = pd.read_csv("../Data/Revised_Training_Set_with_IC50.csv.gz",compression="gzip",header='infer',sep="\t")
test_drug_cell_df = pd.read_csv("../Data/Revised_Test_Set_with_IC50.csv.gz",compression="gzip",header="infer",sep="\t")
print(train_drug_cell_df.shape)
print(test_drug_cell_df.shape)
print(train_drug_cell_df.columns)

#This part of code was not looking at gene expression profiles
#rev_train_feature_df = train_feature_df.iloc[:,[0]+[i for i in range(22844,23322)]]
#rev_test_feature_df = test_feature_df.iloc[:,[0]+[i for i in range(22844,23322)]]

#We now focus on oncogenes, pathway enrichments, module enrichments, mutations in genes, mutation classes
rev_train_feature_df = train_feature_df
rev_test_feature_df = test_feature_df
print(rev_train_feature_df.columns)

(34387, 81)
(19184, 81)
Index(['primary_key', 'dbgap_subject_id', 'dbgap_dnaseq_sample',
       'dbgap_rnaseq_sample', 'inhibitor', 'type', 'status', 'paper_inclusion',
       'min_conc', 'max_conc', 'intercept', 'beta', 'beta_z', 'beta_p', 'aic',
       'pearson_chisq', 'deviance', 'converged', 'ic10', 'ic25', 'ic50',
       'ic75', 'ic90', 'auc', 'all_gt_50', 'all_lt_50', 'curve_type',
       'AUC_[HM] Hypoxia', 'AUC_[HM] Mitotic spindle',
       'AUC_[HM] Wnt beta catenin signaling', 'AUC_[HM] TGF beta signaling',
       'AUC_[HM] DNA repair', 'AUC_[HM] G2M checkpoint', 'AUC_[HM] Apoptosis',
       'AUC_[HM] Notch signaling', 'AUC_[HM] Hedgehog signaling',
       'AUC_[HM] PI3K Akt mTOR signaling', 'AUC_[HM] mTORC1 signaling',
       'AUC_[HM] E2F targets', 'AUC_[HM] Epithelial mesenchymal transition',
       'AUC_[HM] Oxidative phosphorylation', 'AUC_[HM] Glycolysis',
       'AUC_[HM] Reactive oxigen species pathway', 'AUC_[HM] p53 pathway',
       'AUC_[HM] UV response up', 'AUC_[

In [5]:
#Merge the dataframes containing drug-cell info and cell line info df
train_drug_cell_feature_df = pd.merge(train_drug_cell_df, rev_train_feature_df, on="dbgap_rnaseq_sample")
print(train_drug_cell_feature_df.shape)
test_drug_cell_feature_df = pd.merge(test_drug_cell_df, rev_test_feature_df, on="dbgap_rnaseq_sample")
print(test_drug_cell_feature_df.shape)

(34387, 1211)
(19184, 1211)


In [6]:
#Get the drug embedding representation 
drug_embed_df = pd.read_csv("../Data/Drug_Full_SMILES_Embedding.csv",header='infer')
drug_embed_df.rename(columns={"Name":"inhibitor"},inplace=True)
drug_embed_df.head()

#Merge with the drug_cell_feature_df
final_train_drug_feature_cell_feature_df = pd.merge(drug_embed_df, train_drug_cell_feature_df, on = "inhibitor")
print(final_train_drug_feature_cell_feature_df.shape)
final_test_drug_feature_cell_feature_df = pd.merge(drug_embed_df, test_drug_cell_feature_df, on = "inhibitor")
print(final_test_drug_feature_cell_feature_df.shape)

#Write the pickle files
final_train_drug_feature_cell_feature_df.to_pickle("../Data/Training_Set_with_Drug_Embedding_Cell_Info.pkl", compression="zip")
final_test_drug_feature_cell_feature_df.to_pickle("../Data/Test_Set_with_Drug_Embedding_Cell_Info.pkl",compression="zip")

(34387, 1472)
(19184, 1472)


In [10]:
drug_morgan_df = pd.read_csv("../Results/Drugs_file_with_morgan_fps.csv",header='infer')
drug_morgan_df.columns = ["CID","MolecularWeight","CanonicalSMILES","InChIKey","XlogP","inhibitor"]+["MFP"+str(i) for i in range(0,1024)]
drug_morgan_df.head()

#Merge with the drug_cell_feature_df
final_train_drug_mfp_cell_feature_df = pd.merge(drug_morgan_df, train_drug_cell_feature_df, on = "inhibitor")
print(final_train_drug_mfp_cell_feature_df.shape)
final_test_drug_mfp_cell_feature_df = pd.merge(drug_morgan_df, test_drug_cell_feature_df, on = "inhibitor")
print(final_test_drug_mfp_cell_feature_df.shape)

#Write the pickle files
final_train_drug_mfp_cell_feature_df.to_pickle("../Data/Training_Set_with_Drug_MFP_Cell_Info.pkl",compression="zip")
final_test_drug_mfp_cell_feature_df.to_pickle("../Data/Test_Set_with_Drug_MFP_Cell_Info.pkl",compression="zip")

(34387, 2240)
(19184, 2240)
